In [ ]:
!pip install opacus

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.4/254.4 kB 8.1 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import torch.nn.utils.prune as prune
import numpy as np
from opacus import PrivacyEngine
import time
import copy

## Model Hyperparameters

In [ ]:
def select_dataset(name='MNIST'):
  if name == "MNIST":
    transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),])

    full_train = datasets.MNIST(
        root="./data",
        train=True,
        download=True,
        transform=transform
    )

    train_size = 50000
    val_size = len(full_train) - train_size
    train_dataset, val_dataset = random_split(full_train, [train_size, val_size])

    test_dataset = datasets.MNIST(
        root="./data",
        train=False,
        download=True,
        transform=transform
    )

  elif name == "CIFAR":
    transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),])


    full_train = datasets.CIFAR10(
        root="./data",
        train=True,
        download=True,
        transform=transform
    )

    train_size = 40000
    val_size = len(full_train) - train_size
    train_dataset, val_dataset = random_split(full_train, [train_size, val_size])


    test_dataset = datasets.CIFAR10(
        root="./data",
        train=False,
        download=True,
        transform=transform
    )

  train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
  val_loader   = DataLoader(val_dataset, batch_size=256, shuffle=False)
  test_loader  = DataLoader(test_dataset, batch_size=256, shuffle=False)
  return train_loader, val_loader, test_loader

In [26]:
BATCH_SIZE = 128
LR = 0.01
EPOCHS = 20
SEED = 42
MAX_GRAD_NORM = 1.0
DELTA = 1e-5
PATIENCE = 5

DATASET_NAME = "MNIST"
train_loader, val_loader, test_loader = select_dataset(DATASET_NAME)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# For dynamically calculated epsilon
NOISE_MULTIPLIER = 1.0

# For a fixed privacy budget (eps, del)
TARGET_EPSILON = 8
TARGET_DELTA = 1e-5
NUM_EPOCHS = 20

PRUNE_EPOCHS = 2

torch.manual_seed(SEED)
np.random.seed(SEED)

In [ ]:
# baseline model
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        #=======================================#
        #            MNIST Settings
        #=======================================#

        # self.conv1 = nn.Conv2d(1, 16, 3, 1)
        # self.conv2 = nn.Conv2d(16, 32, 3, 1)
        # self.fc1 = nn.Linear(32*12*12, 64)
        # self.fc2 = nn.Linear(64, 10)

        #=======================================#
        #            CIFAR-10 Settings
        #=======================================#

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(in_features=576, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=10)

    def forward(self, x):
        # x = F.relu(self.conv1(x))
        # x = F.relu(self.conv2(x))
        # x = F.max_pool2d(x, 2)
        # x = torch.flatten(x, 1)
        # x = F.relu(self.fc1(x))
        # return self.fc2(x)

        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = torch.flatten(x, 1)

        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [ ]:
# MNIST Model
class CNN_MNIST(nn.Module):
    def __init__(self):
        super().__init__()

        #=======================================#
        #            MNIST Settings
        #=======================================#

        self.conv1 = nn.Conv2d(1, 16, 3, 1)
        self.conv2 = nn.Conv2d(16, 32, 3, 1)
        self.fc1 = nn.Linear(32*12*12, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):

        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)


# CIFAR-10 Model

class CNN_CIFAR10(nn.Module):
    def __init__(self):
        super().__init__()

        #=======================================#
        #            CIFAR-10 Settings
        #=======================================#

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(in_features=576, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=10)

    def forward(self, x):

        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = torch.flatten(x, 1)

        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
def train_one_epoch(model, loader, optimizer):
    # train loop
    model.train()
    total_loss = 0
    for batch_idx, (data, target) in enumerate(loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.cross_entropy(out, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

In [ ]:
@torch.no_grad()
def evaluate(model, loader):
    # evaluation
    model.eval()
    loss, correct = 0, 0
    for data, target in loader:
        data, target = data.to(device), target.to(device)
        out = model(data)
        loss += F.cross_entropy(out, target, reduction="sum").item()
        pred = out.argmax(1)
        correct += pred.eq(target).sum().item()
    loss /= len(loader.dataset)
    acc = 100. * correct / len(loader.dataset)
    return loss, acc

In [ ]:
def weight_pruning(model, amount, return_mask=False, remove=True):
    parameters_to_prune = [
        (m, "weight") for m in model.modules()
        if isinstance(m, (nn.Conv2d, nn.Linear))
    ]

    prune.global_unstructured(
        parameters_to_prune,
        pruning_method=prune.L1Unstructured,
        amount=amount,
    )

    mask_dict = None
    if return_mask:
        mask_dict = {
            f"{name}.weight": module.weight_mask.detach().clone()
            for name, module in model.named_modules()
            if hasattr(module, "weight_mask")
        }

    if remove:
      for module, _ in parameters_to_prune:
          prune.remove(module, "weight")

    total = sum(p.numel() for p in model.parameters())
    zeros = sum((p == 0).sum().item() for p in model.parameters())
    sparsity = 100 * zeros / total

    print(f"Pruned model sparsity = {sparsity:.2f}%")

    return model, mask_dict

In [ ]:
def structured_pruning(model, amount):
    for module in model.modules():
        if isinstance(module, nn.Conv2d):
            prune.ln_structured(
                module,
                name="weight",
                amount=amount,
                n=2,
                dim=0
            )

    for module in model.modules():
        if isinstance(module, nn.Conv2d):
            prune.remove(module, "weight")

    total_channels = 0
    removed_channels = 0
    for module in model.modules():
        if isinstance(module, nn.Conv2d):
            total_channels += module.weight.shape[0]
            removed_channels += (module.weight.abs().sum(dim=(1,2,3)) == 0).sum().item()

    sparsity = 100 * removed_channels / total_channels
    print(f"Structured sparsity: {sparsity:.2f}%")

    return model, None

In [ ]:
def compute_imp_prune_amount(prune_amount, num_iterations):
    remaining = 1 - prune_amount
    p = 1 - remaining ** (1 / num_iterations)
    return p

In [ ]:
def calculate_sparsity(model):
    total = zeros = 0
    for name, p in model.named_parameters():
        if "weight" in name:
            arr = p.detach().cpu().numpy()
            total += arr.size
            zeros += (arr == 0).sum()
    return 100 * zeros / total

In [19]:
def run_experiment(dataset='MNIST', use_dp=False, pruning_type=None, final_sparsity=None, fixed_privacy_budget=True, prune_epochs=None):

    # setup model and optimizer
    if dataset == 'MNIST':
      model = CNN_MNIST().to(device)
    elif dataset == "CIFAR":
      model = CNN_CIFAR10().to(device)

    # Defaults to base CNN class if no dataset is specified
    else:
      model = CNN().to(device)

    optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9)

    if pruning_type == "PRE":
      prune_start_time = time.time()
      model, _ = weight_pruning(model, final_sparsity)
      prune_end_time = time.time()
      prune_time = prune_end_time - prune_start_time
      final_sparsity = calculate_sparsity(model)

    privacy_engine = None
    if use_dp:

        if fixed_privacy_budget:

            privacy_engine = PrivacyEngine()

            # Calculates sigma based on target epsilon and delta
            model, optimizer, train_loader_dp = privacy_engine.make_private_with_epsilon(
              module=model,
              optimizer=optimizer,
              data_loader=train_loader,
              max_grad_norm=MAX_GRAD_NORM,
              target_delta=TARGET_DELTA,
              target_epsilon=TARGET_EPSILON,
              epochs=NUM_EPOCHS
        )
        else:

            privacy_engine = PrivacyEngine()

            # Otherwise takes sigma as a hyperparameter
            model, optimizer, train_loader_dp = privacy_engine.make_private(
              module=model,
              optimizer=optimizer,
              data_loader=train_loader,
              max_grad_norm=MAX_GRAD_NORM,
              noise_multiplier=NOISE_MULTIPLIER,
        )
    else:
        train_loader_dp = train_loader


    # train
    sparsity = 0
    prune_time = 0
    best_val_acc = 0
    epochs_no_improve = 0
    best_model_path = f"best_model{'_dp' if use_dp else ''}.pt"
    rewind_state = None
    global_mask = None
    if pruning_type == "LTH":
      prune_steps = max(1, EPOCHS - 2)
      train_epochs = EPOCHS
      lth_step = 0
    elif pruning_type == "POST":
      train_epochs = EPOCHS - PRUNE_EPOCHS
    else:
      train_epochs = EPOCHS

    start_time = time.time()

    for epoch in range(1, train_epochs + 1):
        train_loss = train_one_epoch(model, train_loader_dp, optimizer)
        val_loss, val_acc = evaluate(model, val_loader)

        eps = privacy_engine.get_epsilon(DELTA)  if use_dp else None

        print(f"[{'DP-SGD' if use_dp else 'Standard SGD'}] Epoch {epoch}: "
            + f"train_loss={train_loss:.4f}, "
            + f"val_loss={val_loss:.4f}, val_acc={val_acc:.4f}%"
            + f", ε={eps:.4f}" if use_dp else "")

        if pruning_type == "LTH" and epoch == 1:
            rewind_state = copy.deepcopy(model.state_dict())
            print("Saved early-rewind weights for LTH")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            epochs_no_improve = 0
            torch.save(model.state_dict(), best_model_path)
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= PATIENCE:
            print(f"\nEarly stopping at epoch {epoch} (no improvement for {PATIENCE} epochs).")
            break

        if pruning_type == "LTH" and epoch != 1 and epoch in prune_epochs:
          lth_step += 1
          remaining_k = (1 - final_sparsity) ** (lth_step / len(prune_epochs))
          target_sparsity = 1 - remaining_k
          print(f"Pruning {target_sparsity*100:.2f}% at epoch {epoch}")
          model, new_mask = weight_pruning(model, amount=target_sparsity, return_mask=True)
          global_mask = new_mask

          rewound = copy.deepcopy(rewind_state)
          for name, mask in global_mask.items():
              if name in rewound:
                  rewound[name] = rewound[name] * mask   # apply lottery ticket mask

          model.load_state_dict(rewound)
          epochs_no_improve = 0

    end_time = time.time()
    total_time = end_time - start_time

    if pruning_type == "POST":
      prune_start_time = time.time()
      model, _ = weight_pruning(model, final_sparsity)
      final_sparsity = calculate_sparsity(model)

      for ft_epoch in range(1, PRUNE_EPOCHS + 1):
            train_loss = train_one_epoch(model, train_loader_dp, optimizer)
            val_loss, val_acc = evaluate(model, val_loader)
            eps = privacy_engine.get_epsilon(DELTA) if use_dp else None

            print(f"[Fine Tuning Epoch {ft_epoch}] val_acc={val_acc:.4f}%"
                  + (f", ε={eps:.4f}" if use_dp else ""))
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                torch.save(model.state_dict(), best_model_path)

      prune_end_time = time.time()
      prune_time = prune_end_time - prune_start_time
      end_time = time.time()
      total_time = end_time - start_time

    # test on best model
    if pruning_type == "LTH":
      # reconstruct final winning ticket
      if global_mask is None:
          ticket_state = copy.deepcopy(rewind_state)
      else:
          ticket_state = copy.deepcopy(rewind_state)
          for name, mask in global_mask.items():
              if name in ticket_state:
                  ticket_state[name] = ticket_state[name] * mask
      model.load_state_dict(ticket_state)
    elif pruning_type == "POST":
      # keep already pruned & fine-tuned model
      pass
    else:
      model.load_state_dict(torch.load(best_model_path))
    test_loss, test_acc = evaluate(model, test_loader)
    final_eps = privacy_engine.get_epsilon(DELTA) if use_dp else None
    final_sparsity = calculate_sparsity(model)

    print(f"\n=== Results for {'DP-SGD' if use_dp else 'Standard-SGD'} with {pruning_type if pruning_type else 'No'} Pruning ===")
    print(f"Test Accuracy: {test_acc:.2f}%")
    print(f"Best Val Acc: {best_val_acc:.2f}%")
    if use_dp:
        print(f"Final ε = {final_eps:.3f}")
    print(f"Train Time:     {total_time:.2f} sec")
    if pruning_type:
        print(f"Sparsity:       {final_sparsity:.2f}%")

    return {
        "val_acc": best_val_acc,
        "test_acc": test_acc,
        "epsilon": final_eps,
        "train_time": total_time,
        "sparsity": final_sparsity
    }

# Baseline: No Differential Privacy

In [23]:
print(f"\nRunning DP-SGD baseline for fixed ({TARGET_EPSILON}, {TARGET_DELTA})-DP")
run_experiment(dataset=DATASET_NAME, use_dp=True, fixed_privacy_budget=True)


Running DP-SGD baseline for fixed (8, 1e-05)-DP


/tmp/ipython-input-1715503071.py:10: UserWarning: Full backward hook is firing when gradients are computed with respect to module outputs since no inputs require gradients. See https://docs.pytorch.org/docs/main/generated/torch.nn.Module.html#torch.nn.Module.register_full_backward_hook for more details.
  loss.backward()


[DP-SGD] Epoch 1: train_loss=2.1974, val_loss=2.0822, val_acc=25.8300%, ε=3.5278
[DP-SGD] Epoch 2: train_loss=2.0328, val_loss=2.0046, val_acc=29.6300%, ε=4.0227
[DP-SGD] Epoch 3: train_loss=1.9762, val_loss=1.9634, val_acc=31.5100%, ε=4.3957
[DP-SGD] Epoch 4: train_loss=1.9503, val_loss=1.9499, val_acc=32.1900%, ε=4.7147
[DP-SGD] Epoch 5: train_loss=1.9309, val_loss=1.9508, val_acc=32.5500%, ε=5.0010
[DP-SGD] Epoch 6: train_loss=1.9272, val_loss=1.9219, val_acc=33.8000%, ε=5.2647
[DP-SGD] Epoch 7: train_loss=1.8984, val_loss=1.9013, val_acc=34.7900%, ε=5.5113
[DP-SGD] Epoch 8: train_loss=1.8821, val_loss=1.8758, val_acc=35.1200%, ε=5.7445
[DP-SGD] Epoch 9: train_loss=1.8556, val_loss=1.8659, val_acc=36.5200%, ε=5.9666
[DP-SGD] Epoch 10: train_loss=1.8482, val_loss=1.8491, val_acc=37.3800%, ε=6.1795
[DP-SGD] Epoch 11: train_loss=1.8079, val_loss=1.8384, val_acc=37.4600%, ε=6.3845
[DP-SGD] Epoch 12: train_loss=1.8191, val_loss=1.8037, val_acc=38.4900%, ε=6.5826
[DP-SGD] Epoch 13: train_

{'val_acc': 41.59,
 'test_acc': 41.88,
 'epsilon': np.float64(7.993863084259577),
 'train_time': 721.7920761108398,
 'sparsity': np.float64(0.0)}

In [20]:
print("\nRunning DP-SGD baseline with pre weight pruning")
run_experiment(dataset=DATASET_NAME, use_dp=True, pruning_type="PRE", final_sparsity=0.3, fixed_privacy_budget=True)


Running DP-SGD baseline with pre weight pruning
Pruned model sparsity = 29.93%


/tmp/ipython-input-1715503071.py:10: UserWarning: Full backward hook is firing when gradients are computed with respect to module outputs since no inputs require gradients. See https://docs.pytorch.org/docs/main/generated/torch.nn.Module.html#torch.nn.Module.register_full_backward_hook for more details.
  loss.backward()


[DP-SGD] Epoch 1: train_loss=2.1971, val_loss=2.0639, val_acc=26.3100%, ε=3.5278
[DP-SGD] Epoch 2: train_loss=2.0120, val_loss=1.9694, val_acc=30.8800%, ε=4.0227
[DP-SGD] Epoch 3: train_loss=1.9464, val_loss=1.9367, val_acc=31.9000%, ε=4.3957
[DP-SGD] Epoch 4: train_loss=1.9171, val_loss=1.9040, val_acc=33.4000%, ε=4.7147
[DP-SGD] Epoch 5: train_loss=1.8927, val_loss=1.8855, val_acc=34.3400%, ε=5.0010
[DP-SGD] Epoch 6: train_loss=1.8739, val_loss=1.8638, val_acc=35.8700%, ε=5.2647
[DP-SGD] Epoch 7: train_loss=1.8476, val_loss=1.8531, val_acc=36.5300%, ε=5.5113
[DP-SGD] Epoch 8: train_loss=1.8313, val_loss=1.8257, val_acc=37.2000%, ε=5.7445
[DP-SGD] Epoch 9: train_loss=1.8217, val_loss=1.8074, val_acc=38.3000%, ε=5.9666
[DP-SGD] Epoch 10: train_loss=1.7982, val_loss=1.8088, val_acc=38.8900%, ε=6.1795
[DP-SGD] Epoch 11: train_loss=1.8019, val_loss=1.8032, val_acc=39.5200%, ε=6.3845
[DP-SGD] Epoch 12: train_loss=1.7875, val_loss=1.7836, val_acc=40.4200%, ε=6.5826
[DP-SGD] Epoch 13: train_

{'val_acc': 44.25,
 'test_acc': 44.84,
 'epsilon': np.float64(7.993863084259577),
 'train_time': 719.7968690395355,
 'sparsity': np.float64(0.0)}

In [21]:
print("\nRunning DP-SGD baseline with post weight pruning")
run_experiment(dataset=DATASET_NAME, use_dp=True, pruning_type="POST", final_sparsity=0.3, fixed_privacy_budget=True)


Running DP-SGD baseline with post weight pruning


/tmp/ipython-input-1715503071.py:10: UserWarning: Full backward hook is firing when gradients are computed with respect to module outputs since no inputs require gradients. See https://docs.pytorch.org/docs/main/generated/torch.nn.Module.html#torch.nn.Module.register_full_backward_hook for more details.
  loss.backward()


[DP-SGD] Epoch 1: train_loss=2.1828, val_loss=2.0609, val_acc=25.6600%, ε=3.5278
[DP-SGD] Epoch 2: train_loss=2.0294, val_loss=2.0015, val_acc=27.7300%, ε=4.0227
[DP-SGD] Epoch 3: train_loss=1.9730, val_loss=1.9422, val_acc=31.0200%, ε=4.3957
[DP-SGD] Epoch 4: train_loss=1.9227, val_loss=1.8925, val_acc=33.4400%, ε=4.7147
[DP-SGD] Epoch 5: train_loss=1.8817, val_loss=1.8511, val_acc=34.9700%, ε=5.0010
[DP-SGD] Epoch 6: train_loss=1.8444, val_loss=1.8582, val_acc=35.6000%, ε=5.2647
[DP-SGD] Epoch 7: train_loss=1.8478, val_loss=1.8315, val_acc=37.0000%, ε=5.5113
[DP-SGD] Epoch 8: train_loss=1.8298, val_loss=1.8111, val_acc=38.1500%, ε=5.7445
[DP-SGD] Epoch 9: train_loss=1.8142, val_loss=1.8281, val_acc=38.6500%, ε=5.9666
[DP-SGD] Epoch 10: train_loss=1.8163, val_loss=1.8037, val_acc=39.4000%, ε=6.1795
[DP-SGD] Epoch 11: train_loss=1.8069, val_loss=1.8111, val_acc=40.1000%, ε=6.3845
[DP-SGD] Epoch 12: train_loss=1.7801, val_loss=1.8086, val_acc=40.8400%, ε=6.5826
[DP-SGD] Epoch 13: train_

{'val_acc': 43.43,
 'test_acc': 43.96,
 'epsilon': np.float64(7.993863084259577),
 'train_time': 724.1310150623322,
 'sparsity': np.float64(0.0)}

In [22]:
print("\nRunning DP-SGD baseline with iterative pruning")
run_experiment(dataset=DATASET_NAME, use_dp=True, pruning_type="LTH", final_sparsity=0.3, fixed_privacy_budget=True, prune_epochs=[5, 10, 15])


Running DP-SGD baseline with iterative pruning


/tmp/ipython-input-1715503071.py:10: UserWarning: Full backward hook is firing when gradients are computed with respect to module outputs since no inputs require gradients. See https://docs.pytorch.org/docs/main/generated/torch.nn.Module.html#torch.nn.Module.register_full_backward_hook for more details.
  loss.backward()


[DP-SGD] Epoch 1: train_loss=2.1913, val_loss=2.0354, val_acc=28.3600%, ε=3.5278
Saved early-rewind weights for LTH
[DP-SGD] Epoch 2: train_loss=1.9857, val_loss=1.9489, val_acc=30.7700%, ε=4.0227
[DP-SGD] Epoch 3: train_loss=1.9226, val_loss=1.9228, val_acc=32.6400%, ε=4.3957
[DP-SGD] Epoch 4: train_loss=1.8925, val_loss=1.8967, val_acc=34.3600%, ε=4.7147
[DP-SGD] Epoch 5: train_loss=1.8764, val_loss=1.8505, val_acc=35.7200%, ε=5.0010
Pruning 11.21% at epoch 5
Pruned model sparsity = 11.18%
[DP-SGD] Epoch 6: train_loss=1.9734, val_loss=1.9432, val_acc=31.4000%, ε=5.2647
[DP-SGD] Epoch 7: train_loss=1.9163, val_loss=1.9130, val_acc=33.3200%, ε=5.5113
[DP-SGD] Epoch 8: train_loss=1.8894, val_loss=1.8932, val_acc=34.5900%, ε=5.7445
[DP-SGD] Epoch 9: train_loss=1.8712, val_loss=1.8602, val_acc=35.6000%, ε=5.9666
[DP-SGD] Epoch 10: train_loss=1.8504, val_loss=1.8455, val_acc=36.5600%, ε=6.1795
Pruning 21.16% at epoch 10
Pruned model sparsity = 21.11%
[DP-SGD] Epoch 11: train_loss=1.9647, v

{'val_acc': 37.25,
 'test_acc': 29.47,
 'epsilon': np.float64(7.993863084259577),
 'train_time': 714.2626428604126,
 'sparsity': np.float64(30.001029018316526)}

MNIST

In [ ]:
print(f"\nRunning DP-SGD baseline for fixed ({TARGET_EPSILON}, {TARGET_DELTA})-DP")
run_experiment(dataset=DATASET_NAME, use_dp=True, fixed_privacy_budget=True)


Running DP-SGD baseline for fixed (8, 1e-05)-DP


/tmp/ipython-input-1715503071.py:10: UserWarning: Full backward hook is firing when gradients are computed with respect to module outputs since no inputs require gradients. See https://docs.pytorch.org/docs/main/generated/torch.nn.Module.html#torch.nn.Module.register_full_backward_hook for more details.
  loss.backward()


[DP-SGD] Epoch 1: train_loss=0.9209, val_loss=0.5501, val_acc=86.9200%, ε=3.6072
[DP-SGD] Epoch 2: train_loss=0.5346, val_loss=0.5889, val_acc=88.6300%, ε=4.1009
[DP-SGD] Epoch 3: train_loss=0.5745, val_loss=0.6062, val_acc=89.2900%, ε=4.4702
[DP-SGD] Epoch 4: train_loss=0.5750, val_loss=0.6006, val_acc=89.9100%, ε=4.7850
[DP-SGD] Epoch 5: train_loss=0.5598, val_loss=0.5834, val_acc=90.3900%, ε=5.0671
[DP-SGD] Epoch 6: train_loss=0.5558, val_loss=0.5712, val_acc=90.9200%, ε=5.3266
[DP-SGD] Epoch 7: train_loss=0.5358, val_loss=0.5597, val_acc=91.3200%, ε=5.5691
[DP-SGD] Epoch 8: train_loss=0.5151, val_loss=0.5546, val_acc=91.4100%, ε=5.7982
[DP-SGD] Epoch 9: train_loss=0.4976, val_loss=0.5380, val_acc=91.7400%, ε=6.0164
[DP-SGD] Epoch 10: train_loss=0.4853, val_loss=0.5150, val_acc=92.0300%, ε=6.2255
[DP-SGD] Epoch 11: train_loss=0.4758, val_loss=0.5104, val_acc=92.2300%, ε=6.4267
[DP-SGD] Epoch 12: train_loss=0.4767, val_loss=0.5070, val_acc=92.2700%, ε=6.6211
[DP-SGD] Epoch 13: train_

{'val_acc': 93.2,
 'test_acc': 93.93,
 'epsilon': np.float64(8.005044757651065),
 'train_time': 3316.64049744606,
 'sparsity': np.float64(0.0)}

In [27]:
print("\nRunning DP-SGD baseline with pre weight pruning")
run_experiment(dataset=DATASET_NAME, use_dp=True, pruning_type="PRE", final_sparsity=0.3, fixed_privacy_budget=True)


Running DP-SGD baseline with pre weight pruning
Pruned model sparsity = 29.99%


/tmp/ipython-input-1715503071.py:10: UserWarning: Full backward hook is firing when gradients are computed with respect to module outputs since no inputs require gradients. See https://docs.pytorch.org/docs/main/generated/torch.nn.Module.html#torch.nn.Module.register_full_backward_hook for more details.
  loss.backward()


[DP-SGD] Epoch 1: train_loss=0.9343, val_loss=0.5505, val_acc=86.1700%, ε=3.6072
[DP-SGD] Epoch 2: train_loss=0.5493, val_loss=0.5734, val_acc=88.3600%, ε=4.1009
[DP-SGD] Epoch 3: train_loss=0.5771, val_loss=0.5851, val_acc=89.2700%, ε=4.4702
[DP-SGD] Epoch 4: train_loss=0.5542, val_loss=0.5802, val_acc=89.8700%, ε=4.7850
[DP-SGD] Epoch 5: train_loss=0.5661, val_loss=0.5744, val_acc=90.2500%, ε=5.0671
[DP-SGD] Epoch 6: train_loss=0.5670, val_loss=0.5486, val_acc=90.6300%, ε=5.3266
[DP-SGD] Epoch 7: train_loss=0.5417, val_loss=0.5444, val_acc=91.0600%, ε=5.5691
[DP-SGD] Epoch 8: train_loss=0.5107, val_loss=0.5313, val_acc=91.1100%, ε=5.7982
[DP-SGD] Epoch 9: train_loss=0.4979, val_loss=0.5152, val_acc=91.5700%, ε=6.0164
[DP-SGD] Epoch 10: train_loss=0.4934, val_loss=0.4914, val_acc=92.0700%, ε=6.2255
[DP-SGD] Epoch 11: train_loss=0.4781, val_loss=0.4917, val_acc=92.0500%, ε=6.4267
[DP-SGD] Epoch 12: train_loss=0.4612, val_loss=0.4848, val_acc=92.2900%, ε=6.6211
[DP-SGD] Epoch 13: train_

{'val_acc': 93.11,
 'test_acc': 94.22,
 'epsilon': np.float64(8.005044757651065),
 'train_time': 3439.1912891864777,
 'sparsity': np.float64(0.0)}

In [ ]:
print("\nRunning DP-SGD baseline with post weight pruning")
run_experiment(dataset=DATASET_NAME, use_dp=True, pruning_type="POST", final_sparsity=0.3, fixed_privacy_budget=True)


Running DP-SGD baseline with post weight pruning


/tmp/ipython-input-1715503071.py:10: UserWarning: Full backward hook is firing when gradients are computed with respect to module outputs since no inputs require gradients. See https://docs.pytorch.org/docs/main/generated/torch.nn.Module.html#torch.nn.Module.register_full_backward_hook for more details.
  loss.backward()


[DP-SGD] Epoch 1: train_loss=1.0450, val_loss=0.5471, val_acc=85.5200%, ε=3.6072
[DP-SGD] Epoch 2: train_loss=0.5385, val_loss=0.5670, val_acc=88.3700%, ε=4.1009
[DP-SGD] Epoch 3: train_loss=0.5504, val_loss=0.5713, val_acc=89.1700%, ε=4.4702
[DP-SGD] Epoch 4: train_loss=0.5532, val_loss=0.5710, val_acc=89.5900%, ε=4.7850
[DP-SGD] Epoch 5: train_loss=0.5531, val_loss=0.5660, val_acc=90.2400%, ε=5.0671
[DP-SGD] Epoch 6: train_loss=0.5318, val_loss=0.5489, val_acc=90.6900%, ε=5.3266
[DP-SGD] Epoch 7: train_loss=0.5339, val_loss=0.5466, val_acc=90.9900%, ε=5.5691
[DP-SGD] Epoch 8: train_loss=0.5150, val_loss=0.5530, val_acc=91.2100%, ε=5.7982
[DP-SGD] Epoch 9: train_loss=0.5218, val_loss=0.5377, val_acc=91.4900%, ε=6.0164
[DP-SGD] Epoch 10: train_loss=0.5046, val_loss=0.5451, val_acc=91.5800%, ε=6.2255
[DP-SGD] Epoch 11: train_loss=0.5021, val_loss=0.5327, val_acc=91.8500%, ε=6.4267


In [ ]:
print("\nRunning DP-SGD baseline with iterative pruning")
run_experiment(dataset=DATASET_NAME, use_dp=True, pruning_type="LTH", final_sparsity=0.3, fixed_privacy_budget=True, prune_epochs=[5, 10, 15])


Running DP-SGD baseline with iterative pruning


/tmp/ipython-input-1715503071.py:10: UserWarning: Full backward hook is firing when gradients are computed with respect to module outputs since no inputs require gradients. See https://docs.pytorch.org/docs/main/generated/torch.nn.Module.html#torch.nn.Module.register_full_backward_hook for more details.
  loss.backward()


[DP-SGD] Epoch 1: train_loss=0.9298, val_loss=0.5437, val_acc=86.6500%, ε=3.6072
Saved early-rewind weights for LTH
[DP-SGD] Epoch 2: train_loss=0.5482, val_loss=0.5643, val_acc=88.6300%, ε=4.1009
[DP-SGD] Epoch 3: train_loss=0.5456, val_loss=0.5878, val_acc=89.3700%, ε=4.4702
[DP-SGD] Epoch 4: train_loss=0.5507, val_loss=0.5872, val_acc=89.8200%, ε=4.7850
[DP-SGD] Epoch 5: train_loss=0.5784, val_loss=0.5845, val_acc=90.1800%, ε=5.0671
Pruning 11.21% at epoch 5
Pruned model sparsity = 11.21%
[DP-SGD] Epoch 6: train_loss=0.5135, val_loss=0.5455, val_acc=88.7600%, ε=5.3266
[DP-SGD] Epoch 7: train_loss=0.5666, val_loss=0.5855, val_acc=89.0500%, ε=5.5691
[DP-SGD] Epoch 8: train_loss=0.5739, val_loss=0.5678, val_acc=89.8900%, ε=5.7982
[DP-SGD] Epoch 9: train_loss=0.5358, val_loss=0.5614, val_acc=90.3400%, ε=6.0164
[DP-SGD] Epoch 10: train_loss=0.5397, val_loss=0.5660, val_acc=90.4400%, ε=6.2255
Pruning 21.16% at epoch 10
Pruned model sparsity = 21.15%
[DP-SGD] Epoch 11: train_loss=0.5100, v